In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Input
import numpy as np
import plotly.graph_objects as go

file_name='TATAMOTORS.csv'
# Load stock price data
data = pd.read_csv(f'./data/raw/{file_name}', parse_dates=True)
data['datetime'] = pd.to_datetime(data['datetime'])

data.set_index('datetime', inplace=True)
data = data[['open', 'high', 'low', 'close', 'volume']]

data

2024-07-12 20:58:02.898923: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-12 20:58:02.994803: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-12 20:58:03.036052: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-12 20:58:03.047420: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-12 20:58:03.118486: I tensorflow/core/platform/cpu_feature_guar

,open,high,low,close,volume
datetime,,,,,
2019-01-01 09:15:00,173.50,173.50,171.30,171.85,1748492
2019-01-01 10:15:00,171.85,172.45,171.80,172.30,592293
2019-01-01 11:15:00,172.30,173.40,172.10,173.25,836524
2019-01-01 12:15:00,173.20,173.45,172.60,173.05,784867
2019-01-01 13:15:00,173.05,173.15,172.55,172.85,725620
...,...,...,...,...,...
2023-08-02 11:15:00,632.90,632.90,626.55,628.20,1438269
2023-08-02 12:15:00,628.20,629.90,623.60,625.05,2168131
2023-08-02 13:15:00,625.00,625.65,616.90,617.90,3091890


In [2]:
# Scale data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Define sequence length for LSTM
sequence_length = 5

# Prepare data for LSTM
x_train, y_train = [], []
for i in range(sequence_length, len(scaled_data)):
    x_train.append(scaled_data[i-sequence_length:i, :])
    y_train.append(scaled_data[i, 0])  # Predict opening price

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], x_train.shape[2]))

# Build LSTM model
model = Sequential()
model.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50))
model.add(Dropout(0.2))
# Adding the output layer
model.add(Dense(25, activation='relu'))  # Predict opening price (Solution 2)
model.add(Dense(units=1))  # Predict opening price (Solution 2)
# Compiling the RNN
model.compile(loss='mse', optimizer='adam')  # Custom loss function
# Fitting the RNN to the Training set
model.fit(x_train, y_train, epochs=50, batch_size=32)


I0000 00:00:1720798084.341079    5570 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1720798084.432341    5570 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1720798084.432502    5570 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1720798084.433644    5570 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Epoch 1/50


2024-07-12 20:58:06.410247: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


295/295 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0557
Epoch 2/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0027
Epoch 3/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0017
Epoch 4/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0015
Epoch 5/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0012
Epoch 6/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0011
Epoch 7/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 8.5715e-04
Epoch 8/50
 40/295 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 6.9112e-04

KeyboardInterrupt: 

In [ ]:
forecast_length = 50

# Forecast future prices
last_x = x_train[-1]
forecast_prices = []
for i in range(forecast_length):
    prediction = model.predict(np.array([last_x]))
    # Prepare the shape for inverse transformation
    prediction_scaled = np.zeros((1, scaled_data.shape[1]))
    prediction_scaled[0, 0] = prediction
    prediction = scaler.inverse_transform(prediction_scaled)
    opening_price = prediction[0][0]
    forecast_prices.append(opening_price)
    last_x = np.append(last_x[1:], prediction_scaled, axis=0)[-sequence_length:]

# Prepare data for visualization
dates = pd.date_range(start=data.index[-1], periods=forecast_length+1, inclusive='right').to_pydatetime()
actual_prices = scaler.inverse_transform(scaled_data[-len(forecast_prices):])[:, 0]


In [ ]:
# Create Plotly graph
fig = go.Figure()
fig.add_trace(go.Scatter(x=dates, y=actual_prices, name='Actual Price'))
fig.add_trace(go.Scatter(x=dates, y=forecast_prices, name='Forecast Price'))
fig.update_layout(title='DLINKINDIA Stock Price Prediction', xaxis_title='Date', yaxis_title='Price')
fig.show()